# 神经网络训练手写识别体

## 准备数据集

In [9]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

In [10]:
batch_size = 64
# 使用torch把图像转化为tensor,图像 c * w * h
# 把单通道转换成多通道，把一系列的图像进行转换，变成 [0,1]之间。0.1307是均值，0.3081是标准差
# 使用Normalize也是希望像素值满足 0 1 分布
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])
# 变换好之后直接放进数据集里
train_dataset = datasets.MNIST(root='./dataset/mnist',train=True,download=True,transform=transform)
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
test_dataset = datasets.MNIST(root='./dataset/mnist',train=False,download=True,transform=transform)
test_loader = DataLoader(test_dataset,shuffle=False,batch_size=batch_size)

## 定义模型

In [11]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.l1 = torch.nn.Linear(784,512)
        self.l2 = torch.nn.Linear(512,256)
        self.l3 = torch.nn.Linear(256,128)
        self.l4 = torch.nn.Linear(128,64)
        self.l5 = torch.nn.Linear(64,10)
        
    def forward(self,x):
        # -1表示会根据剩下的维度计算出数组的另外一个shape属性值
        # 使用relu作为激活函数
        x = x.view(-1,784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))        
        x = F.relu(self.l3(x))        
        x = F.relu(self.l4(x))
        # 最后一个不用经过激活函数，他现在是输出10维的数据
        return self.l5(x)
    
model = Net()

## 构造损失函数和优化器

In [12]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.5)

## 构造训练器

In [16]:
def train(epoch):
    running_loss = 0.0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
        # 优化器在优化之前先清零
        optimizer.zero_grad()
        # forward backward update
        outputs = model(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        # 每300输出一次
        if batch_idx % 300 == 299:
            print('[%d,%5d] loss:%.3f' % (epoch+1,batch_idx+1,running_loss/300))
            # 每300一组数据跑完，清零
            running_loss = 0.0

## 测试

In [20]:
# test里面不需要计算反向传播，只需要计算正向的就行了
def test():
    # 表示正确了多少
    correct = 0
    # 表示总数有多少
    total = 0
    # 在test过程中是不需要计算梯度的
    # 这部分的代码不会求导
    with torch.no_grad():
        for data in test_loader:
            images,labels = data
            outputs = model(images)
            # dim=1 返回每一个行的最大值和索引
            # dim=0 返回每一列的最大值和索引
            # 这里的索引或者下标对应的是它的分类
            _,predicted = torch.max(outputs.data,dim=1)
            # label是一个元组，(N,1)
            # size是计算元素的个数，0表示计算行数，1表示计算列数
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
            
    # 使用正确的数除以总数
    print('Accuracy on test set: %d %%' % (100 * correct/total))

In [21]:
for epoch in range(100):
    # 一轮训练
    train(epoch)
    # 一轮测试
    test()

[1,  300] loss:0.334
[1,  600] loss:0.280
[1,  900] loss:0.229
Accuracy on test set: 94 %
[2,  300] loss:0.183
[2,  600] loss:0.178
[2,  900] loss:0.155
Accuracy on test set: 95 %
[3,  300] loss:0.131
[3,  600] loss:0.121
[3,  900] loss:0.120
Accuracy on test set: 96 %
[4,  300] loss:0.096
[4,  600] loss:0.103
[4,  900] loss:0.088
Accuracy on test set: 96 %
[5,  300] loss:0.073
[5,  600] loss:0.077
[5,  900] loss:0.077
Accuracy on test set: 97 %
[6,  300] loss:0.066
[6,  600] loss:0.061
[6,  900] loss:0.059
Accuracy on test set: 97 %
[7,  300] loss:0.050
[7,  600] loss:0.053
[7,  900] loss:0.050
Accuracy on test set: 97 %
[8,  300] loss:0.038
[8,  600] loss:0.045
[8,  900] loss:0.042
Accuracy on test set: 97 %
[9,  300] loss:0.037
[9,  600] loss:0.030
[9,  900] loss:0.032
Accuracy on test set: 97 %
[10,  300] loss:0.025
[10,  600] loss:0.027
[10,  900] loss:0.028
Accuracy on test set: 97 %
[11,  300] loss:0.018
[11,  600] loss:0.022
[11,  900] loss:0.024
Accuracy on test set: 97 %
[12,

[89,  900] loss:0.000
Accuracy on test set: 98 %
[90,  300] loss:0.000
[90,  600] loss:0.000
[90,  900] loss:0.000
Accuracy on test set: 98 %
[91,  300] loss:0.000
[91,  600] loss:0.000
[91,  900] loss:0.000
Accuracy on test set: 98 %
[92,  300] loss:0.000
[92,  600] loss:0.000
[92,  900] loss:0.000
Accuracy on test set: 98 %
[93,  300] loss:0.000
[93,  600] loss:0.000
[93,  900] loss:0.000
Accuracy on test set: 98 %
[94,  300] loss:0.000
[94,  600] loss:0.000
[94,  900] loss:0.000
Accuracy on test set: 98 %
[95,  300] loss:0.000
[95,  600] loss:0.000
[95,  900] loss:0.000
Accuracy on test set: 98 %
[96,  300] loss:0.000
[96,  600] loss:0.000
[96,  900] loss:0.000
Accuracy on test set: 98 %
[97,  300] loss:0.000
[97,  600] loss:0.000
[97,  900] loss:0.000
Accuracy on test set: 98 %
[98,  300] loss:0.000
[98,  600] loss:0.000
[98,  900] loss:0.000
Accuracy on test set: 98 %
[99,  300] loss:0.000
[99,  600] loss:0.000
[99,  900] loss:0.000
Accuracy on test set: 98 %
[100,  300] loss:0.00